# Clasificación de galaxias

El objetivo de esta tarea es hacer una red neuronal que pueda diferenciar entre tres tipos de galaxia: espiral, irregular y elíptica. Se va a hacer por medio de una red convolucional (para disminuír el número de parámetros de la red.

El ejercicio está basado en el [tutorial](https://www.tensorflow.org/tutorials/layers) de tensorflow para capas en una red convolutiva.

Las fotos de estas galaxias las necesitan sacar del folder zip que se llame 'source' (basta con darle click derecho y luego descomprimir, en realidad.

Primero lo primero, cargar las bibliotecas necesarias. Recuerden, si una les aparece que no está instalada, basta con que utilicen el ambiente virtual para instalar cosas. En particular, tensorflow y PIL (cuyo paquete se llama Pillow, es decir, `pip install pillow`).

In [62]:
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image
import os

Ahora vamos a cargar en RAM todas las fotos. Para ello primero cargamos los nombres y sus tipos (lo único que nos interesa). Me aprovecho de ello para de una vez guardar en un data frame sólo las columnas de interés. No sólo eso pero de una vez vamos a convertir las etiquetas en las tres posibilidades de galaxia que nos interesa clasificar: Elíptica ()

In [63]:
data = pd.read_csv("EFIGI_data.txt")
numToGalaxy = {0: 'Eliptica', 1: 'Espiral', 2: 'Irregular'}

In [64]:
data.head()

,PhotoName,HStage,Type,NType
0,PGC0009530,10,I,2
1,PGC0025524,-5,E,0
2,PGC0002149,-3,S,1
3,PGC0004363,-3,S,1
4,PGC0004540,6,S,1


Esperemos tener una distribución masomenos equitativa de cada galaxia. Usando `groupby` podremos verificar esto

In [65]:
data.groupby("Type").count()

,PhotoName,HStage,NType
Type,,,
E,200,200,200
I,200,200,200
S,200,200,200


Pues... Resulta que no. En fin, A la mejor el problema fue dar un rango tan grande a las espirales

In [66]:
data.groupby("HStage").count().Type

HStage
-6      14
-5     163
-4      23
-3      18
-2      14
-1       7
 0       6
 1      19
 2      14
 3      18
 4      14
 5       8
 6      31
 7      15
 8      27
 9       9
 10    156
 11     44
Name: Type, dtype: int64

Si pues, un poco más homogeneo... Tenemos aún que cargar 82MB de imágenes en RAM. Esto puede ser un grave problema.

In [114]:
# From the dataframe, we make a list of images
images = []

images = data.apply(lambda row: Image.open(os.path.join('source',row.PhotoName+'.png')).convert('L'), axis = 1)
shape = [images[0].height, images[0].width]
# To normalized numpy array
images = np.array(
    list(map(lambda pic: np.reshape(np.array(pic), [1, pic.width * pic.height])[0] , images)), dtype = 'float32'
) / 255
print(images.shape)

(600, 65025)


El siguiente bloque es la parte clave: esriba la estructura de redes convolucionales interna para procesar cada imagen. La última capa y la primer capa ya están hechas. De por medio puede utilizar _pooling_, _flattening_, redes de conexión completa o convoluciones. Como sugerencia: use 2 filtros de 8x8 (convoluciones), luego haga pooling de 3x3 al resultado, luego puede aplanar el resultado y hacer alguna red pequeña de conexión completa con probabilidad de dejar caer algunas aristas.

In [68]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    
    # Input Layer, regresamos las imagenes a su dimensión
    input_layer = tf.reshape(features["x"], [-1, shape[0], shape[1], 1])

    
    
    
    ### AQUÍ TODAS LAS CONEXIONES INTERMEDIAS ###
    lastLayer = tf.contrib.layers.flatten(input_layer)
    
    
    
    # Logits Layer, ternary classification
    logits = tf.layers.dense(inputs=lastLayer, units=3)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Ahora ponemos el estimador. Como nota, el folder tmp se debe borrar para cada nuevo modelo que se haga, o el entrenamiento continuará desde el último valor de los pesos de cada arista.

In [69]:
# Se crea ahora el estimador, en un folder llamado tmp
cs_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./tmp/cs_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/cs_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f56df1eaf98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Aquí se hace el entrenamiento, vaya por un número de repeticiones coherente. El argumento batch_size es el número de fotos que se usarán para calcular el gradiente para cada paso.

In [116]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": images},
    y = np.array([i for i in range(len(images))]),
    batch_size=len(images),
    num_epochs=None,
    shuffle=True)
cs_classifier.train(input_fn=train_input_fn,steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: Received a label value of 599 which is outside the valid range of [0, 3).  Label values: 127 542 51 391 436 583 431 119 153 275 525 425 539 271 380 203 460 141 339 139 243 7 253 371 190 526 588 435 585 43 499 308 365 400 528 504 524 518 92 423 568 56 268 593 334 61 105 122 475 291 343 536 38 404 130 561 599 200 73 538 48 155 471 209 102 146 545 595 193 154 557 414 88 293 75 420 90 304 140 483 439 529 34 177 284 160 444 115 338 192 47 419 448 128 378 541 247 278 480 418 554 458 452 446 28 220 255 508 372 385 292 510 581 223 241 230 166 64 179 14 505 136 537 50 346 183 267 91 468 535 407 23 250 434 490 412 443 5 12 232 199 143 87 472 402 103 59 326 294 313 302 1 488 355 379 516 357 578 316 459 487 579 565 566 65 534 234 411 470 298 198 368 164 562 314 421 405 224 225 82 496 401 486 367 464 558 245 430 438 413 493 495 341 373 67 211 100 381 328 399 58 85 257 273 219 416 533 330 360 258 260 280 492 158 81 180 265 498 342 266 104 222 417 428 395 46 106 522 79 147 594 4

Ahora, para hacer una pequeña prueba y que puedan ver su red funcionando, el siguiente bloque de código carga una imagen de manera aleatoria, y les dice qué tipo de galaxia es (la que su red debería predecir)

In [117]:
idx = np.random.randint(0,600)
testImg = Image.open(os.path.join('source',data.PhotoName[idx]+'.png')).convert('L')
imshow(testImg)
print('La clasificación de la galaxia es ' + numToGalaxy[data.NType[idx]])

La clasificación de la galaxia es Espiral


Este bloque que viene adelante, ejecuta la red para hacer una sola predicción. `pred` es una lista que contiene las probabilidades de cada clase

In [118]:
X = np.reshape(np.array(testImg,dtype = 'float32'), [1, testImg.width * testImg.height])[0] / 255

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {'x': np.reshape(X, [1, -1]).astype('float32')},
    shuffle = False
)
pred = list(cs_classifier.predict(input_fn = predict_input_fn))

print('Predicciones: {}'.format(str(pred)))
print('Debieramos tener una galaxia ' + numToGalaxy[pred[0]['classes']])

ValueError: Could not find trained model in model_dir: ./tmp/cs_model.

# Paso último y más difícil

Busque, google o stackoverflow, cómo hacer una validación cruazada para su red neuronal. Usted tiene 600 fotos, separe 40 de cada grupo para tener 80% de la información para entrenamiento y 20% para hacer una prueba.

Lo puede hacer de manera manual separando la información en 5 bloques, y entrenando con las 5 posibles permutaciones.